In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import json
from datetime import date

In [ ]:
filepath = '/kaggle/input/the-bread-basket/bread basket.csv'
BB = pd.read_csv(filepath)
BB.info()

In [ ]:
BB.head(5)

In [ ]:
BB.describe()

In [ ]:
#function to get the transaction date based on date_time
def getFormattedDate(date_time):
    mydate = date_time.split(' ')[0]
    dateList = mydate.split('-')
    myyear=int(dateList[2])
    mymonth = int(dateList[1])
    myday = int(dateList[0])
    return date(myyear,mymonth,myday)

In [ ]:
#Add one more column to indicate  the date of each transaction
BB['orderDate'] = BB['date_time'].apply(lambda x :getFormattedDate(x) )
BB['orderDate'] 

In [ ]:
#get records by transactions to explore data per transaction
BBTrans= BB[['Transaction','weekday_weekend','date_time','period_day','orderDate']]
BBTrans = BBTrans.drop_duplicates()
BBTrans

In [ ]:
# No. of transactions happened at weekday or weekends
BBTrans['weekday_weekend'].value_counts()

In [ ]:
# percentage of transactions happened at weekday or weekends
BBTrans['weekday_weekend'].value_counts().plot.pie()

In [ ]:
# No. of transactions happened at period of day
BBTrans['period_day'].value_counts()

In [ ]:
# percentage of transactions happened at different period of day
BBTrans['period_day'].value_counts().plot.pie()

In [ ]:
#ranking popularity of different iterm ,  find  top 10 popular goods.
BB['Item'].value_counts().plot.bar(figsize = (20,10))

In [ ]:
#use apyorio to do association analysis:   https://github.com/ymoch/apyori
#install apyorio
!pip install apyori

In [ ]:
from apyori import apriori

In [ ]:
#formulate the list of lists as [['Bread'],[Scandinavian,Scandinavian],['Hot chocolate','Jam','Cookies']...]
# 1. merge the items per each transaction.
# 2. get the list of lists as [['Bread'],[Scandinavian,Scandinavian],['Hot chocolate','Jam','Cookies']...]

BBMergeItemByTrans = BB.groupby('Transaction', as_index=False).agg({'Item' : ','.join}) 
BBMergeItemByTrans

In [ ]:
# 2. get the list of lists as [['Bread'],[Scandinavian,Scandinavian],['Hot chocolate','Jam','Cookies']...]
itemLists = list(BBMergeItemByTrans['Item'])
newLineList = []
for item in itemLists:
    newLine = []
    newLine  = item.split(',')
    newLineList.append(newLine)
newLineList

In [ ]:
#remove duplicated items in each transaction.
uniqueLineList = []
for item in newLineList:
    newLine  = list(set(item))
    uniqueLineList.append(newLine)
uniqueLineList

In [ ]:
# do analysis for support > 1% ,and lift > 1.01
association_rules = apriori(uniqueLineList, min_support=0.01,min_lift = 1.01)

In [ ]:
association_rules= list(association_rules)
association_rules